# Example of data processing using sc2ts, tskit and VCZ

This notebook provides the code for the section of the paper in which we lay out the advantages of the software ecosystem that we are using.

In [1]:
import pathlib
import collections

import sc2ts
import tszip
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
datadir = pathlib.Path("../data")

In [3]:
ds = sc2ts.Dataset(datadir / "viridian_mafft_2024-10-14_v1.vcz.zip")
ds

In [4]:
ts_sc2ts = tszip.load(datadir / "sc2ts_2023-02-21_intersection.trees.tsz")
ts_sc2ts

In [9]:
ts_usher = tszip.load(datadir / "usher_2023-02-21_intersection_ds_di.trees.tsz")
ts_usher

In [10]:
%%time 
dfn_sc2ts = sc2ts.node_data(ts_sc2ts)

CPU times: user 457 ms, sys: 24 ms, total: 481 ms
Wall time: 479 ms


In [11]:
%%time 
dfn_usher = sc2ts.node_data(ts_usher)

CPU times: user 554 ms, sys: 64.2 ms, total: 618 ms
Wall time: 616 ms


Make sure that the two tree sequences we're comparing have the same set of samples and reflects the same set of sequences.

In [12]:
assert np.array_equal(ts_sc2ts.samples(), ts_usher.samples())
assert np.array_equal(dfn_sc2ts[dfn_sc2ts.is_sample].node_id.values, ts_sc2ts.samples())
assert np.array_equal(dfn_usher[dfn_usher.is_sample].node_id.values, ts_usher.samples())
assert np.array_equal(dfn_usher[dfn_usher.is_sample].sample_id.values, dfn_sc2ts[dfn_sc2ts.is_sample].sample_id.values)
assert np.array_equal(ts_sc2ts.sites_position, ts_usher.sites_position)

A key element of processing data efficiently in tskit and VCZ is to use numpy arrays of integers to represent allelic states, instead of the classical approach of using strings, etc. In sc2ts, alleles are given fixed integer representations, such that A=0, C=1, G=2, and T=3. So, to represent the DNA string "AACTG" we would use the numpy array ``[0, 0, 1, 3, 2]`` instead. This has many advantages and makes it much easier to write efficient code. 

The drawback of this is that it's not as easy to inspect and debug, and we must always be aware of the translation required. 

In this analysis we're interested in how well the sc2ts and Usher do at imputing ambiguous bases, and want to count how many times the bases that they impute for samples are compatible with the ambiguity codes.

According to https://www.bioinformatics.org/sms/iupac.html the IUPAC ambiguity codes are as follows:

```
R	G or A	puRine
Y	T or C	pYrimidine
M	A or C	aMino
K	G or T	Keto
S	G or C	Strong interaction (3 H bonds)
W	A or T	Weak interaction (2 H bonds)
H	A or C or T	not-G, H follows G in the alphabet
B	G or T or C	not-A, B follows A
V	G or C or A	not-T (not-U), V follows U
D	G or A or T	not-C, D follows C
```

So, we build up a mapping of each ambiguity code to its compatible bases:

In [13]:
compatible = {
    "R": ["G", "A"],
    "Y": [ "T", "C"], 
   "M": ["A", "C"],
   "K": ["G", "T"],
   "S": ["G", "C"],
   "W": ["A", "T"],
   "H": ["A", "C", "T"],
   "B": ["G", "T", "C"],
   "V": ["G", "C", "A"],
   "D": ["G", "A", "T"],
}

The mapping from alleles to integers is managed by the sc2ts.IUPAC_ALLELES value, and so we build up the corresponding mapping in integer space:

In [19]:
sc2ts.IUPAC_ALLELES

'ACGT-RYSWKMBDHV.'

In [14]:
compatible_encoded = {}
for ambiguity_code, compatible_bases in compatible.items():
    compatible_bases_encoded = [sc2ts.IUPAC_ALLELES.index(base) for base in compatible_bases]
    compatible_encoded[sc2ts.IUPAC_ALLELES.index(ambiguity_code)] = compatible_bases_encoded
compatible_encoded

{5: [2, 0],
 6: [3, 1],
 10: [0, 1],
 9: [2, 3],
 7: [2, 1],
 8: [0, 3],
 13: [0, 1, 3],
 11: [2, 3, 1],
 14: [2, 1, 0],
 12: [2, 0, 3]}

A very important aspect of this data encoding is to understand how missing data is handled. [Explain that it's -1, and that it's better to use -1 and be obvious about it than to hide it with a constant]

In [16]:
sample_id = dfn_usher[dfn_usher.is_sample].sample_id.values
ds_variants = ds.variants(sample_id=sample_id, position=ts_sc2ts.sites_position.astype(int))
usher_variants = ts_usher.variants(alleles=tuple(sc2ts.IUPAC_ALLELES))
sc2ts_variants = ts_sc2ts.variants(alleles=tuple(sc2ts.IUPAC_ALLELES))

iterator = tqdm(zip(ds_variants, usher_variants, sc2ts_variants), total=ts_usher.num_sites)
data = []
for var_ds, var_usher, var_sc2ts in iterator:
    usher_correctly_imputed = 0
    sc2ts_correctly_imputed = 0
    total_ambiguous = 0
    for ambiguity_code, compatible_bases in compatible_encoded.items():
        samples = np.where(var_ds.genotypes == ambiguity_code)[0]
        total_ambiguous += samples.shape[0]
        imputed = collections.Counter(var_usher.genotypes[samples])
        usher_correctly_imputed += sum(imputed[base] for base in compatible_bases)
        imputed = collections.Counter(var_sc2ts.genotypes[samples])
        sc2ts_correctly_imputed += sum(imputed[base] for base in compatible_bases)
    # NOTE: -1 means missing data ("N")
    missing_samples = np.where(var_ds.genotypes == -1)[0]
    imputed_differently = np.sum(
        var_usher.genotypes[missing_samples] != var_sc2ts.genotypes[missing_samples])
       
    data.append({"position": int(var_ds.position), 
                 "total_ambiguous": total_ambiguous, 
                 "usher_correctly_imputed": usher_correctly_imputed,
                 "sc2ts_correctly_imputed": sc2ts_correctly_imputed,
                 "total_missing": missing_samples.shape[0],
                 "imputed_differently": imputed_differently
                })


  0%|          | 0/27431 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(data)
df

,position,total_ambiguous,usher_correctly_imputed,sc2ts_correctly_imputed,total_missing,imputed_differently
0,267,1,1,1,10131,0
1,269,13,13,13,10099,0
2,270,9,9,9,10187,1
3,271,3,3,3,10184,0
4,272,5,5,5,10187,0
...,...,...,...,...,...,...
27426,29667,4,4,4,1122,0
27427,29668,22,22,22,993,0
27428,29669,8,8,8,997,0
27429,29670,28,28,28,1004,2


In [18]:
df["usher_incorrectly_imputed"] = df["total_ambiguous"] - df["usher_correctly_imputed"]
df["sc2ts_incorrectly_imputed"] = df["total_ambiguous"] - df["sc2ts_correctly_imputed"]
df.sum()

position                     407703842
total_ambiguous                 413303
usher_correctly_imputed         413299
sc2ts_correctly_imputed         413226
total_missing                 80574518
imputed_differently              56782
usher_incorrectly_imputed            4
sc2ts_incorrectly_imputed           77
dtype: int64

In [24]:
    77 / 413299 * 100

0.018630579798160653

In [17]:
(df.imputed_differently.sum() / df.total_missing.sum()) * 100

np.float64(0.07047141132138078)

In [22]:
(ts_sc2ts.num_samples * ts_sc2ts.num_sites) / 1024**3

31.335752454586327

The total aligned dataset for 1,226,587 samples and 27,431 sites shared by both the sc2ts and Usher tree sequences represents 31.34GiB of nucleotide calls. Of the 80,574,518 missing data calls (Ns) in the alignments, sc2ts and Usher disagreed in their imputed values for 56,782 (0.07%). Additionally, 413,303 calls made use of the IUPAC uncertainty codes. Of these sc2ts imputed 77 (0.02%) incorrectly (i.e., with a base that is not compatible with the ambiguity code). Remarkably, Usher imputed only 4 calls from this set incorrectly.
Performing this computation by co-iterating over all sites in the source alignments and the sc2ts and Usher tree sequencesrequired only 3:20 seconds on a (computer), with the core logic using Numpy and Pandas requiring around 30 lines of Python code. 